In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "/content/drive/MyDrive/AWS_AI_ML_Scholar"

Mounted at /content/drive
/content/drive/MyDrive/AWS_AI_ML_Scholar


In [1]:
# imports
import os
import requests
from pathlib import Path
import tarfile

# defining dataset directory
data_dir = './flower_data'

# using pathlib.Path for handling PosixPath
FLOWERS_DIR = Path(data_dir)

# downloading and setting up data if not already present
if not FLOWERS_DIR.is_dir():
    # creating directory
    FLOWERS_DIR.mkdir(parents=True, exist_ok=True)
    print(f"[INFO] Directory created: ./{FLOWERS_DIR}")

    print() # for readability

    # tarball path
    TARBALL = FLOWERS_DIR / "flower_data.tar.gz"

    # downloading and writing the tarball to './flowers' directory
    print(f"[INFO] Downloading the file 'flower_data.tar.gz' to ./{FLOWERS_DIR}")
    request = requests.get('https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz')
    with open(TARBALL, "wb") as file_ref:
        file_ref.write(request.content)
        print(f"[INFO] 'flower_data.tar.gz' saved to ./{FLOWERS_DIR}")

    print() # for readability

    # extracting the downloaded tarball
    print(f"[INFO] Extracting the downloaded tarball to ./{FLOWERS_DIR}")
    with tarfile.open(TARBALL, "r") as tar_ref:
        tar_ref.extractall(FLOWERS_DIR)
        print(f"[INFO] 'flower_data.tar.gz' extracted successfully to ./{FLOWERS_DIR}")

    print() # for readability

    # using os.remove to delete the downloaded tarball
    print("[INFO] Deleting the tarball to save space.")
    os.remove(TARBALL)
else:
    print(f"[INFO] Dataset already setup at ./{FLOWERS_DIR}")

[INFO] Directory created: ./flower_data

[INFO] Downloading the file 'flower_data.tar.gz' to ./flower_data
[INFO] 'flower_data.tar.gz' saved to ./flower_data

[INFO] Extracting the downloaded tarball to ./flower_data
[INFO] 'flower_data.tar.gz' extracted successfully to ./flower_data

[INFO] Deleting the tarball to save space.


In [4]:
import json

data = {
    "21": "fire lily", "3": "canterbury bells", "45": "bolero deep blue", "1": "pink primrose", "34": "mexican aster",
    "27": "prince of wales feathers", "7": "moon orchid", "16": "globe-flower", "25": "grape hyacinth", "26": "corn poppy",
    "79": "toad lily", "39": "siam tulip", "24": "red ginger", "67": "spring crocus", "35": "alpine sea holly",
    "32": "garden phlox", "10": "globe thistle", "6": "tiger lily", "93": "ball moss", "33": "love in the mist",
    "9": "monkshood", "102": "blackberry lily", "14": "spear thistle", "19": "balloon flower", "100": "blanket flower",
    "13": "king protea", "49": "oxeye daisy", "15": "yellow iris", "61": "cautleya spicata", "31": "carnation",
    "64": "silverbush", "68": "bearded iris", "63": "black-eyed susan", "69": "windflower", "62": "japanese anemone",
    "20": "giant white arum lily", "38": "great masterwort", "4": "sweet pea", "86": "tree mallow",
    "101": "trumpet creeper", "42": "daffodil", "22": "pincushion flower", "2": "hard-leaved pocket orchid",
    "54": "sunflower", "66": "osteospermum", "70": "tree poppy", "85": "desert-rose", "99": "bromelia", "87": "magnolia",
    "5": "english marigold", "92": "bee balm", "28": "stemless gentian", "97": "mallow", "57": "gaura",
    "40": "lenten rose", "47": "marigold", "59": "orange dahlia", "48": "buttercup", "55": "pelargonium",
    "36": "ruby-lipped cattleya", "91": "hippeastrum", "29": "artichoke", "71": "gazania", "90": "canna lily",
    "18": "peruvian lily", "98": "mexican petunia", "8": "bird of paradise", "30": "sweet william",
    "17": "purple coneflower", "52": "wild pansy", "84": "columbine", "12": "colt's foot", "11": "snapdragon",
    "96": "camellia", "23": "fritillary", "50": "common dandelion", "44": "poinsettia", "53": "primula",
    "72": "azalea", "65": "californian poppy", "80": "anthurium", "76": "morning glory", "37": "cape flower",
    "56": "bishop of llandaff", "60": "pink-yellow dahlia", "82": "clematis", "58": "geranium", "75": "thorn apple",
    "41": "barbeton daisy", "95": "bougainvillea", "43": "sword lily", "83": "hibiscus", "78": "lotus lotus",
    "88": "cyclamen", "94": "foxglove", "81": "frangipani", "74": "rose", "89": "watercress", "73": "water lily",
    "46": "wallflower", "77": "passion flower", "51": "petunia"
}

with open('cat_to_name.json', 'w') as file:
    json.dump(data, file)

In [ ]:
import torch
import argparse
import torchvision.transforms as transforms
from torch import nn, optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
from collections import OrderedDict
from PIL import Image

# Define transformation functions for the datasets
def data_transformations(train_dir, test_dir, valid_dir):
    # Define transformations for training data
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Define transformations for testing and validation data
    test_valid_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load datasets with specified transformations
    train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
    test_data = datasets.ImageFolder(test_dir, transform=test_valid_transforms)
    valid_data = datasets.ImageFolder(valid_dir, transform=test_valid_transforms)

    # Print the number of classes in the dataset for validation
    num_classes = len(train_data.classes)
    print(f"Number of classes in the dataset: {num_classes}")

    # Calculate and print the number of samples in each subset
    num_train_samples = len(train_data)
    num_test_samples = len(test_data)
    num_valid_samples = len(valid_data)
    print(f"Number of training samples: {num_train_samples}")
    print(f"Number of testing samples: {num_test_samples}")
    print(f"Number of validation samples: {num_valid_samples}")

    return train_data, test_data, valid_data



# Define classifier architecture
def classifier_definition(input_size, hidden_units):
    """
    Define the classifier architecture.

    Args:
        input_size (int): The size of the input features.
        hidden_units (int): The number of units in the hidden layer.

    Returns:
        nn.Sequential: The classifier model.
    """

    # Define the structure of the classifier
    classifier = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(input_size, hidden_units)),
        ('relu1', nn.ReLU()),
        ('dropout1', nn.Dropout(0.5)),
        ('fc2', nn.Linear(hidden_units, 102)),  # Output layer with 102 units for 102 classes
        ('output', nn.LogSoftmax(dim=1))
    ]))

    return classifier  # Return the classifier model


# Initialize pre-trained model
def model_initialization(architecture):
    """
    Initialize a pre-trained model based on the specified architecture.

    Args:
        architecture (str): The name of the architecture to use.

    Returns:
        tuple: A tuple containing the initialized model and its input size.
    """

    # Check the specified architecture and initialize the corresponding pre-trained model
    if architecture == 'vgg19':
        model = models.vgg19(pretrained=True)
        input_size = 25088
    elif architecture == 'densenet121':
        model = models.densenet121(pretrained=True)
        input_size = 1024
    elif architecture == 'alexnet':
        model = models.alexnet(pretrained=True)
        input_size = 9216
    else:
        # Raise a ValueError if an invalid model architecture is provided
        raise ValueError("Invalid model architecture")

    # Freeze the parameters of the pre-trained model
    for param in model.parameters():
        param.requires_grad = False

    # Return a tuple containing the initialized model and its input size
    return model, input_size


# Define data loaders
def create_loaders(train_data, test_data, valid_data):
    """
    Create data loaders for the training, testing, and validation datasets.

    Args:
        train_data (Dataset): Training dataset.
        test_data (Dataset): Testing dataset.
        valid_data (Dataset): Validation dataset.

    Returns:
        tuple: A tuple containing the data loaders for training, testing, and validation datasets.
    """
    # Create data loaders for the training, testing, and validation datasets
    valid_loader = DataLoader(valid_data, batch_size=64, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    # Return a tuple containing the data loaders for training, testing, and validation datasets
    return train_loader, test_loader, valid_loader


# Train the model
def model_training(model, train_loader, valid_loader, criterion, optimizer, device, epochs):
    """
    Train the model using the provided data loaders.

    Args:
        model (nn.Module): The model to be trained.
        train_loader (DataLoader): Data loader for the training dataset.
        valid_loader (DataLoader): Data loader for the validation dataset.
        criterion (nn.Module): Loss function criterion.
        optimizer (torch.optim.Optimizer): Optimizer for updating model parameters.
        device (torch.device): Device to be used for training.
        epochs (int): Number of epochs for training.
    """
    print("Training started...")
    model.to(device)
    steps = 0
    print_every = 40
    running_loss = 0

    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            steps += 1

            if steps % print_every == 0:
                validation_loss = 0
                accuracy = 0
                model.eval()

                with torch.no_grad():
                    for inputs, labels in valid_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        logps = model.forward(inputs)
                        batch_loss = criterion(logps, labels)
                        validation_loss += batch_loss.item()
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                print(f"Epoch {epoch + 1}/{epochs}.. "
                      f"Training loss: {running_loss/print_every:.3f}.. "
                      f"Validation loss: {validation_loss/len(valid_loader):.3f}.. "
                      f"Validation accuracy: {accuracy/len(valid_loader):.3f}")
                running_loss = 0
                model.train()

# Test the trained model
def model_testing(model, test_loader, device):
    """
    Test the trained model using the provided test data loader.

    Args:
        model (nn.Module): The trained model to be tested.
        test_loader (DataLoader): Data loader for the test dataset.
        device (torch.device): Device to be used for testing.
    """
    correct = 0
    total = 0
    with torch.no_grad():
        model.eval()
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy on test images: {accuracy:.2f}%")

# Save the trained model checkpoint
def save_model_checkpoint(model, optimizer, input_size, output_size, epochs, save_dir):
    """
    Save the trained model checkpoint to the specified directory.

    Args:
        model (nn.Module): The trained model to be saved.
        optimizer (torch.optim.Optimizer): Optimizer used during training.
        input_size (int): Size of the input layer.
        output_size (int): Size of the output layer.
        epochs (int): Number of epochs trained for.
        save_dir (str): Directory path to save the checkpoint.
    """
    model.class_to_idx = train_data.class_to_idx
    checkpoint = {
        'input_size': input_size,
        'output_size': output_size,
        'hidden_units': args.hidden_units,
        'state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'class_to_idx': model.class_to_idx,
        'epochs': epochs,
        'arch': args.arch
    }
    torch.save(checkpoint, save_dir)
    print(f"Model checkpoint saved to {save_dir}")


# Main function
if __name__ == "__main__":
    # Parse command line arguments
    parser = argparse.ArgumentParser(description="Train a neural network on a custom dataset.")
    parser.add_argument('--data_dir', help='Path to dataset directory', default="flower_data", type=str)
    parser.add_argument('--hidden_units', type=int, dest="hidden_units", action="store", default=120)
    parser.add_argument('--gpu', dest="gpu", action="store", default="gpu")
    parser.add_argument('--epochs', dest="epochs", action="store", type=int, default=10)
    parser.add_argument('--arch', dest="arch", action="store", default="vgg19", type=str, choices=['vgg19', 'densenet121', 'alexnet'])
    parser.add_argument('--save_dir', dest="save_dir", action="store", default="./checkpoint.pth")
    parser.add_argument('--learning_rate', dest="learning_rate", action="store", default=0.001, type=float)
    args = parser.parse_args("")

    train_dir = args.data_dir + '/train'
    test_dir = args.data_dir + '/test'
    valid_dir = args.data_dir + '/valid'

    train_data, test_data, valid_data = data_transformations(train_dir, test_dir, valid_dir)

    train_loader, test_loader, valid_loader = create_loaders(train_data, test_data, valid_data)

    device = torch.device("cuda" if args.gpu == 'gpu' and torch.cuda.is_available() else "cpu")

    model, input_size = model_initialization(args.arch)
    model.classifier = classifier_definition(input_size, args.hidden_units)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=args.learning_rate)

    model_training(model, train_loader, valid_loader, criterion, optimizer, device, args.epochs)

    model_testing(model, test_loader, device)

    save_model_checkpoint(model, optimizer, input_size, 102, args.epochs, args.save_dir)


Number of classes in the dataset: 102
Number of training samples: 6552
Number of testing samples: 819
Number of validation samples: 818
Training started...
